## Test Code

In [1]:
import torch
import copy
import numpy as np
# import evaluate

from datasets import load_dataset, load_metric
from transformers import AutoTokenizer, GPT2ForSequenceClassification, Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
from peft import LoraConfig, get_peft_model

/home/haken/.conda/envs/fedllm/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 加載 IMDB 資料集
# dataset = load_dataset("imdb")
dataset = load_dataset("glue", "mrpc")

# 初始化 GPT-2 的 tokenizer 並設置 padding token
tokenizer = AutoTokenizer.from_pretrained("gpt2", use_fast=True)
tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# 定義 tokenization 函數
def tokenize_function(examples):
    # return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)
    # return tokenizer(examples["sentence1"], examples["sentence2"], padding="max_length", truncation=True, max_length=128)
    return tokenizer(examples["sentence1"], examples["sentence2"], truncation=True, max_length=128)

# 對資料集進行 tokenization
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# 移除無用的列，只保留 input_ids 和 labels
# tokenized_datasets = tokenized_datasets.remove_columns(["text"])
tokenized_datasets = tokenized_datasets.remove_columns(["sentence1", "sentence2", "idx"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")

In [3]:
client_num = 5
client_datasets = [{} for i in range(client_num)]
for data in tokenized_datasets:
    # 获取训练集
    data_dataset = tokenized_datasets[data]
    
    # 假设我们要将训练集拆分为两个子集
    split_ratio = 0.2
    split_index = int(len(data_dataset) * split_ratio)
    
    # 创建两个训练数据集
    for i in range(client_num):
        client_datasets[i][data] = data_dataset.select(range(i*split_index,(i+1)*split_index))

In [4]:
# 設置 LoRA 配置
lora_config = LoraConfig(
    r=8,  # Rank
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
)

# 訓練參數設置
training_args = TrainingArguments(
    output_dir="./llm_models/gpt2_LoRA/server",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_dir="./logs",
)

# 加載 GPT-2 模型並設置 padding token id
model = GPT2ForSequenceClassification.from_pretrained("gpt2", num_labels=2)
model.config.pad_token_id = model.config.eos_token_id

# 使用 LoRA 配置模型
peftmodel = get_peft_model(model, lora_config)
# client_models = [copy.deepcopy(peft_model) for idx in range(client_num)]

/home/haken/.conda/envs/fedllm/lib/python3.9/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/haken/.conda/envs/fedllm/lib/python3.9/site-packages/peft/tuners/lora/layer.py:1119: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [6]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
)

# accuracy_metric = evaluate.load("accuracy")
accuracy_metric = load_metric("accuracy")
predictions = trainer.predict(tokenized_datasets["test"])
pred = np.argmax(predictions.predictions, axis=-1)
accuracy = accuracy_metric.compute(predictions=pred, references=predictions.label_ids)
print(accuracy)

{'accuracy': 0.3356521739130435}


In [7]:
trainer = Trainer(
    model=peftmodel,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
)
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.897600,No log


TrainOutput(global_step=917, training_loss=0.8059116975041746, metrics={'train_runtime': 19.482, 'train_samples_per_second': 188.277, 'train_steps_per_second': 47.069, 'total_flos': 118990871986176.0, 'train_loss': 0.8059116975041746, 'epoch': 1.0})

In [8]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
)
predictions = trainer.predict(tokenized_datasets["test"])
pred = np.argmax(predictions.predictions, axis=-1)
accuracy = accuracy_metric.compute(predictions=pred, references=predictions.label_ids)
print(accuracy)

{'accuracy': 0.6782608695652174}


In [18]:
# load adapter model
from peft import PeftConfig, PeftModel

model = GPT2ForSequenceClassification.from_pretrained("gpt2", num_labels=2)

config = PeftConfig.from_pretrained("./llm_models")
model = PeftModel.from_pretrained(model, "./llm_models")

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
client_weights = [1/client_num for i in range(client_num)]
global_epochs = 2

for epoch in range(global_epochs):
    print(f"------------epoch {epoch}------------")
    for c_idx, c_model in enumerate(client_models):
        print(f"-- client{c_idx} training--")
        
        trainer = Trainer(
            model=c_model,
            args=training_args,
            train_dataset=client_datasets[c_idx]["train"],
            eval_dataset=client_datasets[c_idx]["test"],
            compute_metrics=compute_metrics,
            data_collator=data_collator,
        )
        
        trainer.train()
        # trainer.evaluate()
        evaluation_results = trainer.evaluate()
        print(evaluation_results)

    # aggregate model trainable parameters(adapter)
    with torch.no_grad():
        for key, param in peft_model.named_parameters():
            if param.requires_grad:
                temp = torch.zeros_like(param).cuda()
                for client_idx in range(client_num):
                    temp += client_weights[client_idx] * client_models[client_idx].state_dict()[key]                 
                peft_model.state_dict()[key].data.copy_(temp)
                for client_idx in range(client_num):
                    client_models[client_idx].state_dict()[key].data.copy_(peft_model.state_dict()[key])
    
    for c_idx, c_model in enumerate(client_models):
        trainer = Trainer(
            model=c_model,
            args=training_args,
            train_dataset=client_datasets[c_idx]["train"],
            eval_dataset=client_datasets[c_idx]["test"],
            compute_metrics=compute_metrics,
            # data_collator=data_collator,
        )
        

------------epoch 0------------
-- client0 training--


Epoch,Training Loss,Validation Loss
1,0.004100,No log


{'eval_runtime': 98.544, 'eval_samples_per_second': 50.739, 'eval_steps_per_second': 12.685, 'epoch': 1.0}
-- client1 training--


Epoch,Training Loss,Validation Loss
1,0.021100,No log


{'eval_runtime': 98.7369, 'eval_samples_per_second': 50.64, 'eval_steps_per_second': 12.66, 'epoch': 1.0}
-- client2 training--


...

{'eval_runtime': 98.2637, 'eval_samples_per_second': 50.884, 'eval_steps_per_second': 12.721, 'epoch': 1.0}
-- client3 training--


Epoch,Training Loss,Validation Loss
1,0.015100,No log


{'eval_runtime': 98.2439, 'eval_samples_per_second': 50.894, 'eval_steps_per_second': 12.723, 'epoch': 1.0}
-- client4 training--


Epoch,Training Loss,Validation Loss
1,0.020400,No log


{'eval_runtime': 98.2649, 'eval_samples_per_second': 50.883, 'eval_steps_per_second': 12.721, 'epoch': 1.0}


In [8]:
"""
trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    compute_metrics=compute_metrics,
    # data_collator=data_collator,
)
"""

'\ntrainer = Trainer(\n    model=peft_model,\n    args=training_args,\n    train_dataset=tokenized_datasets["train"],\n    eval_dataset=tokenized_datasets["test"],\n    compute_metrics=compute_metrics,\n    # data_collator=data_collator,\n)\n'

In [9]:
# 開始訓練
# trainer.train()